## libraries and function 


In [ ]:
# !pip install impyute
!pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time
# !pip install missingpy
# from missingpy import MissForest
# note that MissForest uses sklearn.__version__ 0.22.2.post1

## MLE estimation function  




In [ ]:
def diag_term(i,X,y):
  arr0 = X[:,i]
  nar2 = 0
  arr = arr0[~np.isnan(arr0)]
  y_arr = y[~np.isnan(arr0)]

  _, counts = np.unique(y_arr, return_counts=True)
  ind = np.insert(np.cumsum(counts), 0, 0)
  
  return sum([(ind[g]-ind[g-1])*np.var(arr[ind[g-1]:ind[g]]) for 
                       g in range(1,G+1)])/len(y_arr)                       

In [ ]:
def mle(X,y,G):
    '''
    X: input, should be a numpy array
    y: label
    G: number of classes
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    n,p = X.shape[0], X.shape[1]

    # Estimating class means
    mus = np.array([np.nanmean(X[y==g,:],axis=0) for g in range (G)]).T # so that each column is the mean of a class
 
    S = np.diag([diag_term(i,X,y) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        mat = X[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], y[idx]

        _, counts = np.unique(y_arr, return_counts=True)
        ind = np.insert(np.cumsum(counts), 0, 0)

        m_g = counts
 
        A = len(y_arr) 
        scaled_mat = [mat[ind[g-1]:ind[g],:]-mus[[i,j],g-1] for g in range(1,G+1)]   

        q = lambda g: np.dot(scaled_mat[g][:,0],scaled_mat[g][:,0])
        s11 = sum(map(q,range(G))) 
        q = lambda g: np.dot(scaled_mat[g][:,1],scaled_mat[g][:,1])
        s22 = sum(map(q,range(G))) 
        d = lambda g: np.dot(scaled_mat[g][:,0],scaled_mat[g][:,1])
        s12 = sum(map(d,range(G)))  

        start_solve = time.time()
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)

        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0

        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 +
                                      r**2/S[i,i]**2*s11)/condi_var
          # if condi_var <0 then eta = NA. in practice, it's impossible for cov to be negative
          #  therefore, we drop NA elements of eta  
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1:        
            w = [m_g[g-1]*np.cov(mat[ind[g-1]:ind[g],], rowvar=False) for
                 g in range(1,G+1)]
            w = np.sum(w, axis = 0)    
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 
              
        S[i,j] = S[j,i] = r
    return [mus, S]

### compute_err function 
  

In [ ]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
         np.linalg.norm(S_est.flatten()-S.flatten())/S.size]
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape)

In [ ]:
def compute_err_mle(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  running_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    # MLEs approach
    start = time.time()
    mus_mle, S_mle = mle(Xtr_nan,ytrain, G)
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start    
    e_rate.append(mle_err)
    running_time.append(mle_time)
  e_rate = np.asarray(e_rate)
  running_time = np.asarray(running_time)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time)    

In [ ]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([(sum(ytrain==g))*np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_softimpute = np.sum(S_softimpute, axis = 0)/len(ytrain) 
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start

    e_rate.append(softimpute_err)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)    

In [ ]:
def compute_err_mice(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)


    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=100).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([(sum(ytrain==g))*np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_mice = np.sum(S_mice, axis = 0)/len(ytrain) 
    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start

    e_rate.append(mice_err)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)    

# Heart

In [ ]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.train', header = None,sep=',')
test = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.test',
                     header=None, sep = ',')
data = pd.concat([data, test])
data = data.to_numpy()
X,y = data[:,1:], data[:,0]
X = X.astype(np.float32)
G = len(np.unique(y)) 
print(np.shape(X))
for g in range(G):
  print(sum(y==g))

(267, 44)
55
212


In [ ]:
G = 2
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.003, 0.   ],
       [0.004, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.   ]])

In [ ]:
G = 2
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

array([[0.002, 0.   ],
       [0.003, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.   ]])

In [ ]:
G = 2
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

[SoftImpute] Max Singular Value of X_init = 50.945023
[SoftImpute] Iter 1: observed MAE=0.048939 rank=44
[SoftImpute] Iter 2: observed MAE=0.049282 rank=44
[SoftImpute] Iter 3: observed MAE=0.049599 rank=44
[SoftImpute] Iter 4: observed MAE=0.049890 rank=44
[SoftImpute] Iter 5: observed MAE=0.050154 rank=44
[SoftImpute] Iter 6: observed MAE=0.050387 rank=44
[SoftImpute] Iter 7: observed MAE=0.050595 rank=44
[SoftImpute] Iter 8: observed MAE=0.050778 rank=44
[SoftImpute] Iter 9: observed MAE=0.050936 rank=44
[SoftImpute] Iter 10: observed MAE=0.051076 rank=44
[SoftImpute] Iter 11: observed MAE=0.051198 rank=44
[SoftImpute] Iter 12: observed MAE=0.051303 rank=44
[SoftImpute] Iter 13: observed MAE=0.051390 rank=44
[SoftImpute] Iter 14: observed MAE=0.051465 rank=44
[SoftImpute] Iter 15: observed MAE=0.051529 rank=44
[SoftImpute] Iter 16: observed MAE=0.051583 rank=44
[SoftImpute] Iter 17: observed MAE=0.051631 rank=44
[SoftImpute] Iter 18: observed MAE=0.051672 rank=44
[SoftImpute] Iter 1

array([[0.002, 0.   ],
       [0.003, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.001],
       [0.007, 0.001]])


# Inosphere

In [ ]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data',
                  sep = ",", header = None)
# print(data.head())
data = pd.DataFrame.to_numpy(data)
X, y = data[:,:34].astype(np.float64), data[:,34]
le2 = LabelEncoder()
y = le2.fit_transform(y)
print(len(y))
X = np.delete(X,[0,1], axis = 1)
X.shape

351


(351, 32)

In [ ]:
G = 2
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.   ],
       [0.007, 0.001],
       [0.008, 0.001]])

In [ ]:
G = 2
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: Co

array([[0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.   ],
       [0.007, 0.001],
       [0.01 , 0.001]])

In [ ]:
G = 2
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

[SoftImpute] Max Singular Value of X_init = 46.126626
[SoftImpute] Iter 1: observed MAE=0.037561 rank=32
[SoftImpute] Iter 2: observed MAE=0.037606 rank=32
[SoftImpute] Iter 3: observed MAE=0.037645 rank=32
[SoftImpute] Iter 4: observed MAE=0.037677 rank=32
[SoftImpute] Iter 5: observed MAE=0.037704 rank=32
[SoftImpute] Iter 6: observed MAE=0.037725 rank=32
[SoftImpute] Iter 7: observed MAE=0.037742 rank=32
[SoftImpute] Iter 8: observed MAE=0.037754 rank=32
[SoftImpute] Iter 9: observed MAE=0.037762 rank=32
[SoftImpute] Iter 10: observed MAE=0.037767 rank=32
[SoftImpute] Iter 11: observed MAE=0.037767 rank=32
[SoftImpute] Iter 12: observed MAE=0.037764 rank=32
[SoftImpute] Iter 13: observed MAE=0.037758 rank=32
[SoftImpute] Iter 14: observed MAE=0.037749 rank=32
[SoftImpute] Iter 15: observed MAE=0.037739 rank=32
[SoftImpute] Iter 16: observed MAE=0.037726 rank=32
[SoftImpute] Iter 17: observed MAE=0.037713 rank=32
[SoftImpute] Iter 18: observed MAE=0.037699 rank=32
[SoftImpute] Iter 1

array([[0.003, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.001],
       [0.008, 0.   ],
       [0.009, 0.   ]])

# seeds 

In [ ]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt',
                     sep = '\s+', header = None)
data = pd.DataFrame.to_numpy(data)
X,y = data[:,:7], data[:,7]-1 # reset the labels to go start from 0  

In [ ]:
G = 3
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.004, 0.001],
       [0.006, 0.001],
       [0.008, 0.001],
       [0.009, 0.001],
       [0.012, 0.002]])

In [ ]:
G = 3
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: Co

array([[0.004, 0.001],
       [0.007, 0.002],
       [0.013, 0.006],
       [0.016, 0.002],
       [0.023, 0.003]])

In [ ]:
G = 3
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

[SoftImpute] Max Singular Value of X_init = 27.205454
[SoftImpute] Iter 1: observed MAE=0.029558 rank=7
[SoftImpute] Iter 2: observed MAE=0.029637 rank=7
[SoftImpute] Iter 3: observed MAE=0.029718 rank=7
[SoftImpute] Iter 4: observed MAE=0.029802 rank=7
[SoftImpute] Iter 5: observed MAE=0.029886 rank=7
[SoftImpute] Iter 6: observed MAE=0.029972 rank=7
[SoftImpute] Iter 7: observed MAE=0.030058 rank=7
[SoftImpute] Iter 8: observed MAE=0.030144 rank=7
[SoftImpute] Iter 9: observed MAE=0.030235 rank=7
[SoftImpute] Iter 10: observed MAE=0.030329 rank=7
[SoftImpute] Iter 11: observed MAE=0.030421 rank=7
[SoftImpute] Iter 12: observed MAE=0.030514 rank=7
[SoftImpute] Iter 13: observed MAE=0.030616 rank=7
[SoftImpute] Iter 14: observed MAE=0.030714 rank=7
[SoftImpute] Iter 15: observed MAE=0.030811 rank=7
[SoftImpute] Iter 16: observed MAE=0.030911 rank=7
[SoftImpute] Iter 17: observed MAE=0.031019 rank=7
[SoftImpute] Iter 18: observed MAE=0.031132 rank=7
[SoftImpute] Iter 19: observed MAE=0.

array([[0.006, 0.002],
       [0.011, 0.002],
       [0.017, 0.002],
       [0.023, 0.002],
       [0.031, 0.004]])

# wine
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [ ]:
wine = datasets.load_wine()
X,y = wine.data, wine.target.ravel() 
# sum(y==0), sum(y==1), sum(y==2)

In [ ]:
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.005, 0.001],
       [0.007, 0.001],
       [0.009, 0.001],
       [0.01 , 0.001],
       [0.013, 0.002]])

In [ ]:
G = 3
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


array([[0.006, 0.002],
       [0.01 , 0.001],
       [0.012, 0.001],
       [0.016, 0.001],
       [0.021, 0.001]])

In [ ]:
G = 3
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

[SoftImpute] Max Singular Value of X_init = 24.720707
[SoftImpute] Iter 1: observed MAE=0.031373 rank=13
[SoftImpute] Iter 2: observed MAE=0.031426 rank=13
[SoftImpute] Iter 3: observed MAE=0.031476 rank=13
[SoftImpute] Iter 4: observed MAE=0.031525 rank=13
[SoftImpute] Iter 5: observed MAE=0.031573 rank=13
[SoftImpute] Iter 6: observed MAE=0.031617 rank=13
[SoftImpute] Iter 7: observed MAE=0.031659 rank=13
[SoftImpute] Iter 8: observed MAE=0.031698 rank=13
[SoftImpute] Iter 9: observed MAE=0.031735 rank=13
[SoftImpute] Iter 10: observed MAE=0.031769 rank=13
[SoftImpute] Iter 11: observed MAE=0.031800 rank=13
[SoftImpute] Iter 12: observed MAE=0.031829 rank=13
[SoftImpute] Iter 13: observed MAE=0.031855 rank=13
[SoftImpute] Iter 14: observed MAE=0.031879 rank=13
[SoftImpute] Iter 15: observed MAE=0.031901 rank=13
[SoftImpute] Iter 16: observed MAE=0.031922 rank=13
[SoftImpute] Iter 17: observed MAE=0.031942 rank=13
[SoftImpute] Iter 18: observed MAE=0.031960 rank=13
[SoftImpute] Iter 1

array([[0.007, 0.   ],
       [0.012, 0.001],
       [0.016, 0.001],
       [0.021, 0.001],
       [0.027, 0.002]])

# Iris
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [ ]:
iris = datasets.load_iris()
X,y = iris.data, iris.target.ravel() 
G = 3

In [ ]:
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.007, 0.002],
       [0.011, 0.002],
       [0.013, 0.003],
       [0.016, 0.005],
       [0.014, 0.004]])

In [ ]:
G = 3
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:686: Co

array([[0.008, 0.002],
       [0.015, 0.004],
       [0.023, 0.003],
       [0.035, 0.006],
       [0.052, 0.009]])

In [ ]:

G = 3
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

Streaming output truncated to the last 5000 lines.
[SoftImpute] Iter 7: observed MAE=0.024121 rank=4
[SoftImpute] Iter 8: observed MAE=0.024138 rank=4
[SoftImpute] Iter 9: observed MAE=0.024153 rank=4
[SoftImpute] Iter 10: observed MAE=0.024168 rank=4
[SoftImpute] Iter 11: observed MAE=0.024183 rank=4
[SoftImpute] Iter 12: observed MAE=0.024198 rank=4
[SoftImpute] Iter 13: observed MAE=0.024212 rank=4
[SoftImpute] Iter 14: observed MAE=0.024226 rank=4
[SoftImpute] Iter 15: observed MAE=0.024239 rank=4
[SoftImpute] Iter 16: observed MAE=0.024251 rank=4
[SoftImpute] Iter 17: observed MAE=0.024264 rank=4
[SoftImpute] Iter 18: observed MAE=0.024276 rank=4
[SoftImpute] Iter 19: observed MAE=0.024287 rank=4
[SoftImpute] Iter 20: observed MAE=0.024297 rank=4
[SoftImpute] Iter 21: observed MAE=0.024307 rank=4
[SoftImpute] Iter 22: observed MAE=0.024318 rank=4
[SoftImpute] Iter 23: observed MAE=0.024329 rank=4
[SoftImpute] Iter 24: observed MAE=0.024344 rank=4
[SoftImpute] Iter 25: observed MAE

array([[0.013, 0.002],
       [0.026, 0.005],
       [0.04 , 0.004],
       [0.054, 0.007],
       [0.066, 0.005]])